## Import Library

In [1]:
import cv2
import os
from ultralytics import YOLO

## Data

### กำหนดcategory id ที่เราต้องการและ กรองjsonออกมาแค่ตัวที่เราต้องการ

In [ ]:
import json

# กำหนดค่า category_id ที่ต้องการกรอง
ids = {0, 4, 7}

# อ่านข้อมูลจากไฟล์ data.json
with open('test_labels_200.json', 'r') as file:
    data = json.load(file)

# กรอง annotations ที่มี category_id อยู่ใน ids
filtered_annotations = [anno for anno in data['annotations'] if anno['category_id'] in ids]

# กรอง images ที่มี image_id ตรงกับ annotations ที่คัดเลือก
filtered_image_ids = {anno['image_id'] for anno in filtered_annotations}
filtered_images = [img for img in data['images'] if img['id'] in filtered_image_ids]

# กรอง categories ที่มี id อยู่ใน ids
filtered_categories = [cat for cat in data['categories'] if cat['id'] in ids]

# สร้างข้อมูลใหม่ที่มีแค่ category_id ที่เรากำหนด
filtered_data = {
    "info": data['info'],
    "licenses": data['licenses'],
    "categories": filtered_categories,
    "images": filtered_images,
    "annotations": filtered_annotations
}

# บันทึกข้อมูลที่กรองแล้วลงไฟล์ใหม่
output_file = 'filtered.json'
with open(output_file, 'w') as outfile:
    json.dump(filtered_data, outfile, indent=4)

print(f"กรองข้อมูลสำเร็จ! ข้อมูลที่กรองถูกบันทึกในไฟล์ {output_file}")


### เก็บรูปภาพที่ตรงกับข้อมูลที่เราต้องการให้เรียนรู้

In [ ]:
import json
import os
import shutil

# ชื่อโฟลเดอร์ที่ใช้เก็บไฟล์ภาพ
source_dir = 'data'
target_dir = 'data2'

# ตรวจสอบว่ามีโฟลเดอร์ data2 หรือยัง ถ้ายังไม่มีให้สร้างใหม่
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# อ่านข้อมูลจาก filtered.json
with open('filtered.json', 'r') as file:
    labels2 = json.load(file)

# ดึงชื่อไฟล์ภาพที่ต้องการจาก filtered
target_files = {image['file_name'] for image in labels2['images']}

# คัดลอกภาพที่ตรงกับ target_files ไปยัง data2
copied_count = 0
for file_name in os.listdir(source_dir):
    if file_name in target_files:
        source_path = os.path.join(source_dir, file_name)
        target_path = os.path.join(target_dir, file_name)
        shutil.copy2(source_path, target_path)
        copied_count += 1

print(f"คัดลอกภาพสำเร็จทั้งหมด {copied_count} ภาพไปยังโฟลเดอร์ '{target_dir}'!")


### แปลง format json ให้อยู่ใน format yolo

In [ ]:
import json
import os

# ชื่อไฟล์และโฟลเดอร์ และกำหนดlabelที่เราต้องการของclassนั้น
input_file = 'filtered.json' # กำหนดinput json ที่ต้องการแปลง
output_dir = 'new_labels' #โฟลเดอร์output
id = "1" # กำหนดlabels

# ตรวจสอบว่ามีโฟลเดอร์ new_labels หรือยัง ถ้ายังไม่มีให้สร้างใหม่
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# อ่านข้อมูลจาก input_file
with open(input_file, 'r') as file:
    data = json.load(file)

# ดึงขนาดของภาพจากข้อมูล images
image_info = {img['id']: img for img in data['images']}

# ดึง class id จาก categories
category_mapping = {cat['id']: idx for idx, cat in enumerate(data['categories'])}

# แปลงข้อมูล annotations เป็น YOLO format
for annotation in data['annotations']:
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    bbox = annotation['bbox']

    # ดึงขนาดภาพ
    img_width = image_info[image_id]['width']
    img_height = image_info[image_id]['height']

    # แปลงข้อมูล bbox เป็น YOLO format (normalized)
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    norm_width = width / img_width
    norm_height = height / img_height

    # สร้างบรรทัดข้อมูลใน YOLO format
    yolo_line = f"{id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}"

    # ชื่อไฟล์ .txt สำหรับเก็บ YOLO label
    file_name = image_info[image_id]['file_name'].replace('.jpg', '.txt')
    output_file = os.path.join(output_dir, file_name)

    # บันทึกข้อมูลลงไฟล์
    with open(output_file, 'a') as out_file:
        out_file.write(yolo_line + '\n')

print(f"แปลงข้อมูลสำเร็จ! YOLO labels ถูกบันทึกในโฟลเดอร์ '{output_dir}'")


### กรณีที่ข้อมูลไม่อยู่ในรูปแบบที่เราต้องการ 


#### เราจะเอาภาพมาแปลงเป็นjpg แล้วแปลงชื่อไฟล์ให้อยู่ในรูปแบบ yolo

In [ ]:
from PIL import Image
import os

# ระบุโฟลเดอร์ที่มีไฟล์ต้นฉบับ
source_folder = "class0_sharpk"
destination_folder = "converted_images"
name ="etc" #กำหนดชื่อนำหน้าไฟล์ภาพที่เราต้องการ 

# สร้างโฟลเดอร์ปลายทางถ้ายังไม่มี
os.makedirs(destination_folder, exist_ok=True)

# วนลูปเพื่อแปลงไฟล์ทั้งหมดในโฟลเดอร์
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        # ดึงเลขดั้งเดิมจากชื่อไฟล์ เช่น 0000 จาก 0000.png
        original_number = os.path.splitext(filename)[0]
        
        # เปิดไฟล์ภาพ
        img_path = os.path.join(source_folder, filename)
        img = Image.open(img_path)
        
        # แปลงไฟล์เป็น JPG และบันทึกด้วยชื่อใหม่
        new_filename = f"{name}_{original_number}.jpg"
        new_path = os.path.join(destination_folder, new_filename)
        img.convert("RGB").save(new_path, "JPEG")
        
        print(f"แปลงไฟล์ {filename} -> {new_filename} เสร็จสิ้น")

print("แปลงไฟล์ทั้งหมดเสร็จสิ้น!")


### กรณีภาพที่ไม่มีการตรวจจับแล้วต้องการสร้างlabels เราจะใช้ตัวนี้ในการสร้าง

In [ ]:
import os

# ระบุโฟลเดอร์ที่มีไฟล์ภาพ
image_folder = "converted_images"
# ระบุโฟลเดอร์ที่เราต้องการให้เก็บไฟล์ .txt
txt_folder = "labels_train_etc"

# สร้างโฟลเดอร์ปลายทางสำหรับไฟล์ .txt ถ้ายังไม่มี
os.makedirs(txt_folder, exist_ok=True)

# วนลูปเพื่อสร้างไฟล์ .txt สำหรับทุกภาพในโฟลเดอร์
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        # ดึงชื่อไฟล์โดยไม่เอานามสกุล เช่น etc_0000 จาก etc_0000.jpg
        file_name_without_ext = os.path.splitext(filename)[0]
        
        # สร้างไฟล์ .txt ในโฟลเดอร์ labels_train_etc
        txt_file_path = os.path.join(txt_folder, f"{file_name_without_ext}.txt")
        
        # เขียนข้อมูล "3 0.5 0.5 1.0 1.0" ลงในไฟล์ .txt
        with open(txt_file_path, 'w') as f:
            f.write("3 0.5 0.5 1.0 1.0\n")
        
        print(f"สร้างไฟล์ {file_name_without_ext}.txt และเขียนค่า 3 0.5 0.5 1.0 1.0")

print("สร้างไฟล์ .txt ทั้งหมดเสร็จสิ้น!")

### แปลงlabel json ไปformat yolo สำหรับที่มีหลายclassและต้องการกำหนดclass id ให้เป็นตามที่เรต้องการ

In [ ]:
import json
import os

# กำหนด mapping ใหม่สำหรับ category_id ตามที่ระบุ
category_mapping = {
    0: 2,  # Ascaris lumbricoides → 2
    4: 1,  # Hookworm egg → 1
    7: 0   # Opisthorchis viverrine → 0
}

# อ่านข้อมูลจากไฟล์ labels.json
with open('filtered_0_4_7.json', 'r') as file:
    data = json.load(file)

# สร้างไฟล์ classes.txt
classes = ["Opisthorchis viverrine", "Hookworm egg", "Ascaris lumbricoides"]
with open('classes.txt', 'w') as f:
    for cls in classes:
        f.write(cls + "\n")

# สร้างไดเรกทอรีสำหรับเก็บไฟล์ .txt ในโฟลเดอร์ new_labels
os.makedirs('new_labels', exist_ok=True)

# สร้างไฟล์ .txt สำหรับแต่ละภาพในโฟลเดอร์ new_labels
for image in data['images']:
    image_id = image['id']
    image_width = image['width']
    image_height = image['height']
    label_file = f"new_labels/{image['file_name'].replace('.jpg', '.txt')}"
    
    # ค้นหา annotation ที่ตรงกับภาพนี้
    annotations = [anno for anno in data.get('annotations', []) if anno['image_id'] == image_id]
    
    with open(label_file, 'w') as f:
        for anno in annotations:
            category_id = anno['category_id']
            # ตรวจสอบว่ามี mapping หรือไม่
            if category_id in category_mapping:
                new_category_id = category_mapping[category_id]
                
                # ดึงข้อมูล bbox
                x, y, w, h = anno['bbox']
                
                # คำนวณ normalized coordinates
                x_center = (x + w / 2) / image_width
                y_center = (y + h / 2) / image_height
                norm_width = w / image_width
                norm_height = h / image_height
                
                # เขียนข้อมูลลงไฟล์
                f.write(f"{new_category_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n")

print("แปลงข้อมูลเป็น YOLO format สำเร็จ! ไฟล์ถูกบันทึกในโฟลเดอร์ new_labels")

## Preprocessing

In [ ]:
'''มีการใช้ Matlab ในการ Preprcessing ในเรื่อง Unshapmasking หรือ การทำภาพมีให้ความคมขึ้น เนื่องจากภาพที่ได้มามีความเบลอ จึงจำเป็นต้องมีการเพิ่มความเข็มของเส้นขอบเพื่อลดความเบลอ'''
"""
imageFolder = 'D:\AB-BiMaGOoOD\Tob-taun\3rdYears\Y3T2\ComputerVision\Pre-processingImageDL\data_train\';
outputFolder = 'D:\AB-BiMaGOoOD\Tob-taun\3rdYears\Y3T2\ComputerVision\Pre-processingImageDL\destination_store';
imageFiles = dir(fullfile(imageFolder, '*.jpg'));
numImage = numel(imageFiles);

for i = 1:numImage
    img_get = imread(fullfile(imageFolder, imageFiles(i).name));

    % Convert to double precision for processing
    im = im2double(img_get);
    
    % Apply median filtering to each channel
    b1 = medfilt2(im(:, :, 1), [13, 13]);
    b2 = medfilt2(im(:, :, 2), [13, 13]);
    b3 = medfilt2(im(:, :, 3), [13, 13]);
    blur_image = cat(3, b1, b2, b3);
    
    % Compute the edge image
    edge_image = im - blur_image;

    % Enhance the image by adding the edge image
    output_image = im + (4.5 * edge_image);
    
    % Save the processed image
    outputFile = fullfile(outputFolder, imageFiles(i).name);
    imwrite(output_image, outputFile);
end
"""

"\nimageFolder = 'D:\\AB-BiMaGOoOD\\Tob-taun\x03rdYears\\Y3T2\\ComputerVision\\Pre-processingImageDL\\data_train';\noutputFolder = 'D:\\AB-BiMaGOoOD\\Tob-taun\x03rdYears\\Y3T2\\ComputerVision\\Pre-processingImageDL\\destination_store';\nimageFiles = dir(fullfile(imageFolder, '*.jpg'));\nnumImage = numel(imageFiles);\n\nfor i = 1:numImage\n    img_get = imread(fullfile(imageFolder, imageFiles(i).name));\n\n    % Convert to double precision for processing\n    im = im2double(img_get);\n    \n    % Apply median filtering to each channel\n    b1 = medfilt2(im(:, :, 1), [13, 13]);\n    b2 = medfilt2(im(:, :, 2), [13, 13]);\n    b3 = medfilt2(im(:, :, 3), [13, 13]);\n    blur_image = cat(3, b1, b2, b3);\n    \n    % Compute the edge image\n    edge_image = im - blur_image;\n\n    % Enhance the image by adding the edge image\n    output_image = im + (4.5 * edge_image);\n    \n    % Save the processed image\n    outputFile = fullfile(outputFolder, imageFiles(i).name);\n    imwrite(output_image

## Model Training

In [2]:
# โหลดโมเดล
model = YOLO("yolov8n.pt")

In [3]:
# เทรนโมเดล
history = model.train(data='../dataset/data.yaml',
                      epochs=15,
                      imgsz=640,
                      save=True,
                      project='D:\\AB-BiMaGOoOD\\DiddyLearning\\model\\runs\\detect',
                      name='train',)

New https://pypi.org/project/ultralytics/8.3.87 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.86  Python-3.10.16 torch-2.7.0.dev20250308+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning D:\AB-BiMaGOoOD\DiddyLearning\dataset\labels\train.cache... 2299 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2299/2299 [00:00<?, ?it/s]
val: Scanning D:\AB-BiMaGOoOD\DiddyLearning\dataset\labels\val.cache... 594 images, 0 backgrounds, 0 corrupt: 100%|██████████| 594/594 [00:00<?, ?it/s]


Plotting labels to D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/15      2.01G      1.006      2.573     0.9984         13        640: 100%|██████████| 144/144 [01:27<00:00,  1.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:16<00:00,  1.16it/s]


                   all        594        596      0.932      0.797      0.906      0.687

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15      1.99G     0.5737     0.3547     0.8621         11        640: 100%|██████████| 144/144 [01:18<00:00,  1.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  6.29it/s]


                   all        594        596      0.986      0.987      0.992      0.867

15 epochs completed in 0.399 hours.
Optimizer stripped from D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2\weights\best.pt, 6.2MB

Validating D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2\weights\best.pt...
Ultralytics 8.3.86  Python-3.10.16 torch-2.7.0.dev20250308+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.53it/s]


                   all        594        596      0.986      0.987      0.992      0.866
 Opisthochis_viverrine        200        200       0.99      0.981       0.99      0.844
          Hookworm_egg        194        195      0.973      0.985       0.99      0.817
  Ascaris_lumbricoides        200        201      0.995      0.995      0.995      0.935
Speed: 0.3ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\train2


## Evaluate Model

In [4]:
model = YOLO("D:\\AB-BiMaGOoOD\\DiddyLearning\\model\\runs\\detect\\train2\\weights\\best.pt")  # load a custom model

# Validate the model
metrics = model.val(imgsz=640, project='D:\\AB-BiMaGOoOD\\DiddyLearning\\model\\runs\\detect\\val', conf=0.313)
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics 8.3.86  Python-3.10.16 torch-2.7.0.dev20250308+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\AB-BiMaGOoOD\DiddyLearning\dataset\labels\val.cache... 594 images, 0 backgrounds, 0 corrupt: 100%|██████████| 594/594 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:17<00:00,  2.13it/s]


                   all        594        596      0.987      0.987      0.991      0.878
 Opisthochis_viverrine        200        200       0.99       0.98      0.988      0.855
          Hookworm_egg        194        195      0.975      0.985       0.99      0.839
  Ascaris_lumbricoides        200        201      0.995      0.995      0.995       0.94
Speed: 0.3ms preprocess, 9.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to D:\AB-BiMaGOoOD\DiddyLearning\model\runs\detect\val\val2


array([     0.8549,     0.83866,     0.94041])

In [5]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020708A6F7C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

## Predicted by Model

In [10]:
predict_image_01 = 'D:\\AB-BiMaGOoOD\\DiddyLearning\\dataset\\images\\test\\Ascaris lumbricoides_0991.jpg'
predict_image_02 = 'D:\\AB-BiMaGOoOD\\DiddyLearning\\dataset\\images\\test\Hookworm egg_0914.jpg'
predict_image_03 = 'D:\\AB-BiMaGOoOD\\DiddyLearning\\dataset\\images\\test\\Opisthorchis viverrine_0991.jpg'
predict_image_04 = 'D:\\AB-BiMaGOoOD\\DiddyLearning\\dataset\\images\\test\\no_egg.jpg'

model = YOLO("../model/runs/detect/train2/weights/best.pt")

results = model([predict_image_01, predict_image_02, predict_image_03, predict_image_04], conf= 0.342)

for result in results:
    boxes = result.boxes
    masks = result.masks
    keypoints = result.keypoints
    probs = result.probs
    obb = result.obb
    result.show()


0: 640x640 1 Ascaris_lumbricoides, 3.7ms
1: 640x640 1 Hookworm_egg, 3.7ms
2: 640x640 1 Opisthochis_viverrine, 3.7ms
3: 640x640 (no detections), 3.7ms
Speed: 2.9ms preprocess, 3.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [7]:
# Load trained model
model = YOLO("../model/runs/detect/train2/weights/best.pt")  # Update path if needed

# Run inference on a test image
results = model.predict(
    "../dataset/images/test",
    imgsz=640, 
    save=True,
    project="D:\\AB-BiMaGOoOD\\DiddyLearning\\model\\runs\\detect",
    name="predicted",
    conf=0.342)

print("Inference complete! Check the 'runs/detect' folder for results.")


image 1/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0991.jpg: 608x640 1 Ascaris_lumbricoides, 55.7ms
image 2/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0992.jpg: 640x480 1 Ascaris_lumbricoides, 98.9ms
image 3/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0993.jpg: 640x480 1 Ascaris_lumbricoides, 13.5ms
image 4/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0994.jpg: 640x352 1 Ascaris_lumbricoides, 45.1ms
image 5/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0995.jpg: 608x640 1 Ascaris_lumbricoides, 9.4ms
image 6/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0996.jpg: 640x480 1 Ascaris_lumbricoides, 9.2ms
image 7/30 d:\AB-BiMaGOoOD\DiddyLearning\Scripts\..\dataset\images\test\Ascaris lumbricoides_0997.jpg: 640x480 1 Ascaris_lumbricoides, 8.5ms
image 8/